In [1]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
import time
import datetime
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

In [2]:
matches_23_24 = pd.read_csv("data/teams_pl_23-24_fbref.csv")
matches_22_23 = pd.read_csv("data/teams_pl_22-23_fbref.csv")
matches_21_22 = pd.read_csv("data/teams_pl_21-22_fbref.csv")
matches_20_21 = pd.read_csv("data/teams_pl_20-21_fbref.csv")
matches_19_20 = pd.read_csv("data/teams_pl_19-20_fbref.csv")
matches_18_19 = pd.read_csv("data/teams_pl_18-19_fbref.csv")
matches = pd.concat([matches_23_24, matches_22_23, matches_21_22, matches_20_21, matches_19_20, matches_18_19], ignore_index=True)
stadiums = pd.read_csv("data/stadiums.csv")

In [3]:
matches.head(2)

,season,venue_date,venue_time,round,attendance_value,referee,home_manager,away_manager,home_captain,away_captain,...,away_fouls,away_fouled,away_offsides,away_pens_won,away_pens_conceded,away_own_goals,away_ball_recoveries,away_aerials_won,away_aerials_lost,away_aerials_won_pct
0,2023-2024,2023-08-11,20:00,1,21572.0,Craig Pawson,Vincent Kompany,Pep Guardiola,Josh Cullen,Kevin De Bruyne,...,8,11,1,0,0,0,54,13,13,50.0
1,2023-2024,2023-08-12,12:30,1,59984.0,Michael Oliver,Mikel Arteta,Steve Cooper,Martin Ødegaard,Joe Worrall,...,12,12,1,0,0,0,34,20,12,62.5


In [4]:
matches.rename(columns={"venue_date": "date", "venue_time": "time"}, inplace=True)
matches = matches.drop(columns = ["home_shirtnumber", "home_nationality", "home_position", "home_age", "away_shirtnumber", "away_nationality", "away_position", "away_age"])

In [5]:
matches["home_goals"] = matches["home_goals"] + matches["away_own_goals"]
matches["away_goals"] = matches["away_goals"] + matches["home_own_goals"]
matches["total_goals"] = matches["home_goals"] + matches["away_goals"]

#### Add stadium info to every match

In [6]:
matches_with_stadiums = pd.merge(matches, stadiums[stadiums["Closed"].isna()], left_on=['home_team'], right_on=['Club'], how='left').drop(columns=["Closed", "Opened"])
matches_with_stadiums.head(2)

,season,date,time,round,attendance_value,referee,home_manager,away_manager,home_captain,away_captain,...,away_aerials_won_pct,total_goals,Stadium,Club,Location,Capacity,Pitch length m,Pitch width m,latitude,longitude
0,2023-2024,2023-08-11,20:00,1,21572.0,Craig Pawson,Vincent Kompany,Pep Guardiola,Josh Cullen,Kevin De Bruyne,...,50.0,3,Turf Moor,Burnley,Burnley,21744.0,105.0,68.0,53°47′21″N,2°13′49″W
1,2023-2024,2023-08-12,12:30,1,59984.0,Michael Oliver,Mikel Arteta,Steve Cooper,Martin Ødegaard,Joe Worrall,...,62.5,3,Emirates Stadium,Arsenal,London,60704.0,105.0,68.0,51°33′18″N,000°06′31″W


#### Add columns with match outcome and points of each team

In [7]:
def determine_outcome_and_points(row):
    if row['home_goals'] > row['away_goals']:
        return 1, 3, 0  # Home win
    elif row['home_goals'] < row['away_goals']:
        return 2, 0, 3  # Away win
    else:
        return 0, 1, 1  # Draw  
    
matches_with_stadiums[['outcome', 'home_points', 'away_points']] = matches_with_stadiums.apply(lambda row: pd.Series(determine_outcome_and_points(row)), axis=1)

In [8]:
pd.set_option('display.max_columns', None)
matches_with_stadiums.head(2)

,season,date,time,round,attendance_value,referee,home_manager,away_manager,home_captain,away_captain,formation_home,formation_away,home_team,away_team,home_minutes,home_goals,home_assists,home_pens_made,home_pens_att,home_shots,home_shots_on_target,home_cards_yellow,home_cards_red,home_touches,home_tackles,home_interceptions,home_blocks,home_xg,home_npxg,home_xg_assist,home_sca,home_gca,home_passes_completed,home_passes,home_passes_pct,home_progressive_passes,home_carries,home_progressive_carries,home_take_ons,home_take_ons_won,home_passes_total_distance,home_passes_progressive_distance,home_passes_completed_short,home_passes_short,home_passes_pct_short,home_passes_completed_medium,home_passes_medium,home_passes_pct_medium,home_passes_completed_long,home_passes_long,home_passes_pct_long,home_pass_xa,home_assisted_shots,home_passes_into_final_third,home_passes_into_penalty_area,home_crosses_into_penalty_area,home_passes_live,home_passes_dead,home_passes_free_kicks,home_through_balls,home_passes_switches,home_crosses,home_throw_ins,home_corner_kicks,home_corner_kicks_in,home_corner_kicks_out,home_corner_kicks_straight,home_passes_offsides,home_passes_blocked,home_tackles_won,home_tackles_def_3rd,home_tackles_mid_3rd,home_tackles_att_3rd,home_challenge_tackles,home_challenges,home_challenge_tackles_pct,home_challenges_lost,home_blocked_shots,home_blocked_passes,home_tackles_interceptions,home_clearances,home_errors,home_touches_def_pen_area,home_touches_def_3rd,home_touches_mid_3rd,home_touches_att_3rd,home_touches_att_pen_area,home_touches_live_ball,home_take_ons_won_pct,home_take_ons_tackled,home_take_ons_tackled_pct,home_carries_distance,home_carries_progressive_distance,home_carries_into_final_third,home_carries_into_penalty_area,home_miscontrols,home_dispossessed,home_passes_received,home_progressive_passes_received,home_cards_yellow_red,home_fouls,home_fouled,home_offsides,home_pens_won,home_pens_conceded,home_own_goals,home_ball_recoveries,home_aerials_won,home_aerials_lost,home_aerials_won_pct,away_minutes,away_goals,away_assists,away_pens_made,away_pens_att,away_shots,away_shots_on_target,away_cards_yellow,away_cards_red,away_touches,away_tackles,away_interceptions,away_blocks,away_xg,away_npxg,away_xg_assist,away_sca,away_gca,away_passes_completed,away_passes,away_passes_pct,away_progressive_passes,away_carries,away_progressive_carries,away_take_ons,away_take_ons_won,away_passes_total_distance,away_passes_progressive_distance,away_passes_completed_short,away_passes_short,away_passes_pct_short,away_passes_completed_medium,away_passes_medium,away_passes_pct_medium,away_passes_completed_long,away_passes_long,away_passes_pct_long,away_pass_xa,away_assisted_shots,away_passes_into_final_third,away_passes_into_penalty_area,away_crosses_into_penalty_area,away_passes_live,away_passes_dead,away_passes_free_kicks,away_through_balls,away_passes_switches,away_crosses,away_throw_ins,away_corner_kicks,away_corner_kicks_in,away_corner_kicks_out,away_corner_kicks_straight,away_passes_offsides,away_passes_blocked,away_tackles_won,away_tackles_def_3rd,away_tackles_mid_3rd,away_tackles_att_3rd,away_challenge_tackles,away_challenges,away_challenge_tackles_pct,away_challenges_lost,away_blocked_shots,away_blocked_passes,away_tackles_interceptions,away_clearances,away_errors,away_touches_def_pen_area,away_touches_def_3rd,away_touches_mid_3rd,away_touches_att_3rd,away_touches_att_pen_area,away_touches_live_ball,away_take_ons_won_pct,away_take_ons_tackled,away_take_ons_tackled_pct,away_carries_distance,away_carries_progressive_distance,away_carries_into_final_third,away_carries_into_penalty_area,away_miscontrols,away_dispossessed,away_passes_received,away_progressive_passes_received,away_cards_yellow_red,away_fouls,away_fouled,away_offsides,away_pens_won,away_pens_conceded,away_own_goals,away_ball_recoveries,away_aerials_won,away_aerials_lost,away_aerials_won_pct,total_goals,Stadium,Club,Location,Capacity,Pitch length m,Pitch width m,latitude,lo

### Create dynamic table updated after every match played

In [18]:
cols = matches_with_stadiums.columns
attributes = []
for col in cols:
    if col.startswith("home_"):
        attributes.append(col.replace("home_", ""))
attributes = attributes[3:] # delete team, captain and manager attributes
# attributes consists of only number columns - not nominal columns

In [19]:
standings_columns = ['matches_played', 'wins', 'draws', 'defeats', 'goal_difference', 'goals_conceded'] + attributes
standings = pd.DataFrame(columns=['season', 'team'] + standings_columns)
matches_sorted = matches_with_stadiums.sort_values(by=['season', 'date'])

In [24]:
for season in matches_sorted['season'].unique():
    season_data = matches_sorted[matches_sorted['season'] == season]
    teams = season_data['home_team'].unique()
    
    standings_dict = {team: {attribute: 0 for attribute in standings_columns} for team in teams}

    # tu można dodać punkty karne, z którymi drużyny zaczynają sezon, bardzo niewiele to zmieni ale jednak w teorii dokładniejszy model!!!!!!
    
    for dt in season_data['date'].unique():
        round_data = season_data[season_data['date'] == dt]
        
        for index, row in round_data.iterrows():
            home_team = row['home_team']
            away_team = row['away_team']
            home_goals = row['home_goals']
            away_goals = row['away_goals']
            home_points = row['home_points']
            away_points = row['away_points']
            outcome = row['outcome']
            
            # Update home team standings
            standings_dict[home_team]['matches_played'] += 1
            standings_dict[home_team]['goals_conceded'] += away_goals
            
            for attr in attributes:
                standings_dict[home_team][attr] += row["home_" + attr]

            standings_dict[home_team]['goal_difference'] = standings_dict[home_team]['goals'] - standings_dict[home_team]['goals_conceded']
            
            # Update away team standings
            standings_dict[away_team]['matches_played'] += 1
            standings_dict[away_team]['goals_conceded'] += home_goals
            
            for attr in attributes:
                standings_dict[away_team][attr] += row["away_" + attr]

            standings_dict[away_team]['goal_difference'] = standings_dict[away_team]['goals'] - standings_dict[away_team]['goals_conceded']

            # Update W L D
            if outcome == 1:
                standings_dict[home_team]['wins'] += 1
                standings_dict[away_team]['defeats'] += 1
            elif outcome == 0:
                standings_dict[home_team]['draws'] += 1
                standings_dict[away_team]['draws'] += 1
            else:
                standings_dict[home_team]['defeats'] += 1
                standings_dict[away_team]['wins'] += 1

        temp_standings_data = {
        'season': season,
        'date': dt,
        'team': list(standings_dict.keys()),
        'matches_played': [standings_dict[team]['matches_played'] for team in standings_dict],
        'goals_conceded': [standings_dict[team]['goals_conceded'] for team in standings_dict],
        'goal_difference': [standings_dict[team]['goal_difference'] for team in standings_dict],
        'wins': [standings_dict[team]['wins'] for team in standings_dict],
        'draws': [standings_dict[team]['draws'] for team in standings_dict],
        'defeats': [standings_dict[team]['defeats'] for team in standings_dict]}


        for attr in attributes:
            temp_standings_data[attr] = [standings_dict[team][attr] for team in standings_dict]

        temp_standings = pd.DataFrame(temp_standings_data)

        standings = pd.concat([standings, temp_standings])
standings = standings.sort_values(by=['season', 'date', 'points', 'goal_difference', 'goals'], ascending=[True, True, False, False, False])
standings.reset_index(drop=True, inplace=True)

In [25]:
standings[standings["matches_played"] == 38].head(2)

,season,team,matches_played,wins,draws,defeats,goal_difference,goals_conceded,minutes,goals,assists,pens_made,pens_att,shots,shots_on_target,cards_yellow,cards_red,touches,tackles,interceptions,blocks,xg,npxg,xg_assist,sca,gca,passes_completed,passes,passes_pct,progressive_passes,carries,progressive_carries,take_ons,take_ons_won,passes_total_distance,passes_progressive_distance,passes_completed_short,passes_short,passes_pct_short,passes_completed_medium,passes_medium,passes_pct_medium,passes_completed_long,passes_long,passes_pct_long,pass_xa,assisted_shots,passes_into_final_third,passes_into_penalty_area,crosses_into_penalty_area,passes_live,passes_dead,passes_free_kicks,through_balls,passes_switches,crosses,throw_ins,corner_kicks,corner_kicks_in,corner_kicks_out,corner_kicks_straight,passes_offsides,passes_blocked,tackles_won,tackles_def_3rd,tackles_mid_3rd,tackles_att_3rd,challenge_tackles,challenges,challenge_tackles_pct,challenges_lost,blocked_shots,blocked_passes,tackles_interceptions,clearances,errors,touches_def_pen_area,touches_def_3rd,touches_mid_3rd,touches_att_3rd,touches_att_pen_area,touches_live_ball,take_ons_won_pct,take_ons_tackled,take_ons_tackled_pct,carries_distance,carries_progressive_distance,carries_into_final_third,carries_into_penalty_area,miscontrols,dispossessed,passes_received,progressive_passes_received,cards_yellow_red,fouls,fouled,offsides,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct,points,date
4280,2018-2019,Manchester City,38,32,2,4,72,23,37618,95,71,3,4,679,252,44,1,32238,518,362,319,85.7,82.7,65.8,1216,168,24678,28389,3293.1,2429,22457,1325,734,437,395041,113549,12249,13188,3521.6,9318,10379,3405.4,1907,2834,2555.4,63.4,516,1948,554,62,26660,1631,344,112,226,783,739,298,85,121,1,98,384,327,214,215,89,193,467,1565.9,274,63,256,880,570,15,1825,6098,16583,9800,1458,32234,2252.0,296,1544.1,118990,68197,1017,366,509,393,24519,2412,0,328,301,98,4,4,0,2000,540,522,1958.8,98,2019-05-12
4281,2018-2019,Manchester City,38,32,2,4,72,23,37618,95,71,3,4,679,252,44,1,32238,518,362,319,85.7,82.7,65.8,1216,168,24678,28389,3293.1,2429,22457,1325,734,437,395041,113549,12249,13188,3521.6,9318,10379,3405.4,1907,2834,2555.4,63.4,516,1948,554,62,26660,1631,344,112,226,783,739,298,85,121,1,98,384,327,214,215,89,193,467,1565.9,274,63,256,880,570,15,1825,6098,16583,9800,1458,32234,2252.0,296,1544.1,118990,68197,1017,366,509,393,24519,2412,0,328,301,98,4,4,0,2000,540,522,1958.8,98,2019-05-12


In [34]:
matches_filtered = matches_with_stadiums[matches_with_stadiums['round'] > 3]
df = matches_filtered.copy()
df.head(2)

,season,date,time,round,attendance_value,referee,home_manager,away_manager,home_captain,away_captain,formation_home,formation_away,home_team,away_team,home_minutes,home_goals,home_assists,home_pens_made,home_pens_att,home_shots,home_shots_on_target,home_cards_yellow,home_cards_red,home_touches,home_tackles,home_interceptions,home_blocks,home_xg,home_npxg,home_xg_assist,home_sca,home_gca,home_passes_completed,home_passes,home_passes_pct,home_progressive_passes,home_carries,home_progressive_carries,home_take_ons,home_take_ons_won,home_passes_total_distance,home_passes_progressive_distance,home_passes_completed_short,home_passes_short,home_passes_pct_short,home_passes_completed_medium,home_passes_medium,home_passes_pct_medium,home_passes_completed_long,home_passes_long,home_passes_pct_long,home_pass_xa,home_assisted_shots,home_passes_into_final_third,home_passes_into_penalty_area,home_crosses_into_penalty_area,home_passes_live,home_passes_dead,home_passes_free_kicks,home_through_balls,home_passes_switches,home_crosses,home_throw_ins,home_corner_kicks,home_corner_kicks_in,home_corner_kicks_out,home_corner_kicks_straight,home_passes_offsides,home_passes_blocked,home_tackles_won,home_tackles_def_3rd,home_tackles_mid_3rd,home_tackles_att_3rd,home_challenge_tackles,home_challenges,home_challenge_tackles_pct,home_challenges_lost,home_blocked_shots,home_blocked_passes,home_tackles_interceptions,home_clearances,home_errors,home_touches_def_pen_area,home_touches_def_3rd,home_touches_mid_3rd,home_touches_att_3rd,home_touches_att_pen_area,home_touches_live_ball,home_take_ons_won_pct,home_take_ons_tackled,home_take_ons_tackled_pct,home_carries_distance,home_carries_progressive_distance,home_carries_into_final_third,home_carries_into_penalty_area,home_miscontrols,home_dispossessed,home_passes_received,home_progressive_passes_received,home_cards_yellow_red,home_fouls,home_fouled,home_offsides,home_pens_won,home_pens_conceded,home_own_goals,home_ball_recoveries,home_aerials_won,home_aerials_lost,home_aerials_won_pct,away_minutes,away_goals,away_assists,away_pens_made,away_pens_att,away_shots,away_shots_on_target,away_cards_yellow,away_cards_red,away_touches,away_tackles,away_interceptions,away_blocks,away_xg,away_npxg,away_xg_assist,away_sca,away_gca,away_passes_completed,away_passes,away_passes_pct,away_progressive_passes,away_carries,away_progressive_carries,away_take_ons,away_take_ons_won,away_passes_total_distance,away_passes_progressive_distance,away_passes_completed_short,away_passes_short,away_passes_pct_short,away_passes_completed_medium,away_passes_medium,away_passes_pct_medium,away_passes_completed_long,away_passes_long,away_passes_pct_long,away_pass_xa,away_assisted_shots,away_passes_into_final_third,away_passes_into_penalty_area,away_crosses_into_penalty_area,away_passes_live,away_passes_dead,away_passes_free_kicks,away_through_balls,away_passes_switches,away_crosses,away_throw_ins,away_corner_kicks,away_corner_kicks_in,away_corner_kicks_out,away_corner_kicks_straight,away_passes_offsides,away_passes_blocked,away_tackles_won,away_tackles_def_3rd,away_tackles_mid_3rd,away_tackles_att_3rd,away_challenge_tackles,away_challenges,away_challenge_tackles_pct,away_challenges_lost,away_blocked_shots,away_blocked_passes,away_tackles_interceptions,away_clearances,away_errors,away_touches_def_pen_area,away_touches_def_3rd,away_touches_mid_3rd,away_touches_att_3rd,away_touches_att_pen_area,away_touches_live_ball,away_take_ons_won_pct,away_take_ons_tackled,away_take_ons_tackled_pct,away_carries_distance,away_carries_progressive_distance,away_carries_into_final_third,away_carries_into_penalty_area,away_miscontrols,away_dispossessed,away_passes_received,away_progressive_passes_received,away_cards_yellow_red,away_fouls,away_fouled,away_offsides,away_pens_won,away_pens_conceded,away_own_goals,away_ball_recoveries,away_aerials_won,away_aerials_lost,away_aerials_won_pct,total_goals,Stadium,Club,Location,Capacity,Pitch length m,Pitch width m,latitude,lo

### Liczenie agregacji statystyk na bazie tabeli

In [35]:
# Function to get team stats
def get_team_stats(season, team, matches_played):
    return standings[(standings["season"] == season) & (standings["team"] == team) & (standings["matches_played"] == matches_played)].iloc[0]


def calculate_rolling_stats(n, stats, stats_old, matches_played, real_matches_played):
    rolling_stats = {}
    for feature in standings_columns:
        if matches_played == 0:
            rolling_stats[feature] = stats[feature]
            if real_matches_played != 0:
                rolling_stats[feature] = rolling_stats[feature] / real_matches_played
        else:
            rolling_stats[feature] = stats[feature] - stats_old[feature]
            rolling_stats[feature] = rolling_stats[feature] / n
    return rolling_stats

def create_rolling_stats(n, df):
    for index, row in df.iterrows():
        date = row["date"]
        season = row["season"]
        home_team = row["home_team"]
        away_team = row["away_team"]

        home_team_matches_played = standings[(standings["date"] == date) & (standings["team"] == home_team)]["matches_played"].values[0] - 1
        away_team_matches_played = standings[(standings["date"] == date) & (standings["team"] == away_team)]["matches_played"].values[0] - 1

        home_team_stats = get_team_stats(season, home_team, home_team_matches_played)
        away_team_stats = get_team_stats(season, away_team, away_team_matches_played)

        home_team_matches_played_old = max(0, home_team_matches_played - n)
        away_team_matches_played_old = max(0, away_team_matches_played - n)

        if home_team_matches_played_old > 0:
            home_team_stats_old = get_team_stats(season, home_team, home_team_matches_played_old)
        if away_team_matches_played_old > 0:
            away_team_stats_old = get_team_stats(season, away_team, away_team_matches_played_old)

        home_rolling_stats = calculate_rolling_stats(n, home_team_stats, home_team_stats_old if home_team_matches_played_old > 0 else None, home_team_matches_played_old, home_team_matches_played)
        away_rolling_stats = calculate_rolling_stats(n, away_team_stats, away_team_stats_old if away_team_matches_played_old > 0 else None, away_team_matches_played_old, away_team_matches_played)

        for feature, value in home_rolling_stats.items():
            df.at[index, f"home_last{n}_{feature}"] = value
        for feature, value in away_rolling_stats.items():
            df.at[index, f"away_last{n}_{feature}"] = value

        for feature in ['wins', 'draws', 'defeats', 'goal_difference', 'goals_conceded']:
            if feature in standings_columns:
                df.at[index, f"home_last{n}_{feature}"] = home_team_stats[feature]
                df.at[index, f"away_last{n}_{feature}"] = away_team_stats[feature]
                if home_team_matches_played > 0:
                    df.at[index, f"home_last{n}_{feature}"] = home_team_stats[feature] / home_team_matches_played
                if away_team_matches_played > 0:
                    df.at[index, f"away_last{n}_{feature}"] = away_team_stats[feature] / away_team_matches_played

In [38]:
pd.options.mode.chained_assignment = None
create_rolling_stats(n = 5, df = df)

In [39]:
df.head(2)

,season,date,time,round,attendance_value,referee,home_manager,away_manager,home_captain,away_captain,formation_home,formation_away,home_team,away_team,home_minutes,home_goals,home_assists,home_pens_made,home_pens_att,home_shots,home_shots_on_target,home_cards_yellow,home_cards_red,home_touches,home_tackles,home_interceptions,home_blocks,home_xg,home_npxg,home_xg_assist,home_sca,home_gca,home_passes_completed,home_passes,home_passes_pct,home_progressive_passes,home_carries,home_progressive_carries,home_take_ons,home_take_ons_won,home_passes_total_distance,home_passes_progressive_distance,home_passes_completed_short,home_passes_short,home_passes_pct_short,home_passes_completed_medium,home_passes_medium,home_passes_pct_medium,home_passes_completed_long,home_passes_long,home_passes_pct_long,home_pass_xa,home_assisted_shots,home_passes_into_final_third,home_passes_into_penalty_area,home_crosses_into_penalty_area,home_passes_live,home_passes_dead,home_passes_free_kicks,home_through_balls,home_passes_switches,home_crosses,home_throw_ins,home_corner_kicks,home_corner_kicks_in,home_corner_kicks_out,home_corner_kicks_straight,home_passes_offsides,home_passes_blocked,home_tackles_won,home_tackles_def_3rd,home_tackles_mid_3rd,home_tackles_att_3rd,home_challenge_tackles,home_challenges,home_challenge_tackles_pct,home_challenges_lost,home_blocked_shots,home_blocked_passes,home_tackles_interceptions,home_clearances,home_errors,home_touches_def_pen_area,home_touches_def_3rd,home_touches_mid_3rd,home_touches_att_3rd,home_touches_att_pen_area,home_touches_live_ball,home_take_ons_won_pct,home_take_ons_tackled,home_take_ons_tackled_pct,home_carries_distance,home_carries_progressive_distance,home_carries_into_final_third,home_carries_into_penalty_area,home_miscontrols,home_dispossessed,home_passes_received,home_progressive_passes_received,home_cards_yellow_red,home_fouls,home_fouled,home_offsides,home_pens_won,home_pens_conceded,home_own_goals,home_ball_recoveries,home_aerials_won,home_aerials_lost,home_aerials_won_pct,away_minutes,away_goals,away_assists,away_pens_made,away_pens_att,away_shots,away_shots_on_target,away_cards_yellow,away_cards_red,away_touches,away_tackles,away_interceptions,away_blocks,away_xg,away_npxg,away_xg_assist,away_sca,away_gca,away_passes_completed,away_passes,away_passes_pct,away_progressive_passes,away_carries,away_progressive_carries,away_take_ons,away_take_ons_won,away_passes_total_distance,away_passes_progressive_distance,away_passes_completed_short,away_passes_short,away_passes_pct_short,away_passes_completed_medium,away_passes_medium,away_passes_pct_medium,away_passes_completed_long,away_passes_long,away_passes_pct_long,away_pass_xa,away_assisted_shots,away_passes_into_final_third,away_passes_into_penalty_area,away_crosses_into_penalty_area,away_passes_live,away_passes_dead,away_passes_free_kicks,away_through_balls,away_passes_switches,away_crosses,away_throw_ins,away_corner_kicks,away_corner_kicks_in,away_corner_kicks_out,away_corner_kicks_straight,away_passes_offsides,away_passes_blocked,away_tackles_won,away_tackles_def_3rd,away_tackles_mid_3rd,away_tackles_att_3rd,away_challenge_tackles,away_challenges,away_challenge_tackles_pct,away_challenges_lost,away_blocked_shots,away_blocked_passes,away_tackles_interceptions,away_clearances,away_errors,away_touches_def_pen_area,away_touches_def_3rd,away_touches_mid_3rd,away_touches_att_3rd,away_touches_att_pen_area,away_touches_live_ball,away_take_ons_won_pct,away_take_ons_tackled,away_take_ons_tackled_pct,away_carries_distance,away_carries_progressive_distance,away_carries_into_final_third,away_carries_into_penalty_area,away_miscontrols,away_dispossessed,away_passes_received,away_progressive_passes_received,away_cards_yellow_red,away_fouls,away_fouled,away_offsides,away_pens_won,away_pens_conceded,away_own_goals,away_ball_recoveries,away_aerials_won,away_aerials_lost,away_aerials_won_pct,total_goals,Stadium,Club,Location,Capacity,Pitch length m,Pitch width m,latitude,lo

In [ ]:
df.to_csv("data/matches_with_rolling_stats_pl.csv", index=False)